In [125]:
import requests
import json
import lxml.html as lh
from lxml.html import fromstring

rest_ws_uri = "https://uts-ws.nlm.nih.gov/rest"
version = "current"

def get_tgt(apikey):
    p = {'apikey': apikey}
    h = {"Content-type": "application/x-www-form-urlencoded", "Accept": "text/plain", "User-Agent":"python" }
    r = requests.post('https://utslogin.nlm.nih.gov/cas/v1/api-key',data=p,headers=h)
    return fromstring(r.text).xpath('//form/@action')[0]

def get_st(tgt):
    p = {'service': 'http://umlsks.nlm.nih.gov'}
    h = {"Content-type": "application/x-www-form-urlencoded", "Accept": "text/plain", "User-Agent":"python" }
    r = requests.post(tgt,data=p,headers=h)
    return r.text

def get_query_result(service, query, raw_url=False):
    st = get_st(tgt)
    query['ticket'] = st
    url = service if raw_url else rest_ws_uri + service
    r = requests.get(url, params=query)
    r.encoding = 'utf-8'
    data = json.loads(r.text)
    return data['result']

def search(query):
    return get_query_result('/search/' + version, query)

def definition(query, CUI):
    return get_query_result('/content/' + version + '/CUI/' + CUI + '/definitions', query)

def information(query, CUI):
    return get_query_result('/content/' + version + '/CUI/' + CUI, query)

def atoms(query, CUI):
    return get_query_result('/content/' + version + '/CUI/' + CUI + '/atoms', query)

def parents(query, source_id):
    return get_query_result(source_id + '/parents', query, raw_url=True)

def children(query, source_id):
    return get_query_result(source_id + '/children', query, raw_url=True)

def relations(query, source_id):
    return get_query_result(source_id + '/relations', query, raw_url=True)


api_key = "...." # Fill the string with your API key generated by NLM's website. 
tgt = get_tgt(api_key)
print(tgt)

https://utslogin.nlm.nih.gov/cas/v1/api-key/TGT-1245436-Czc633gI4QuI399mb7p6U9KYMMtcczTAeSgPAnPS2ZkMmaexIv-cas


In [51]:
# Search for "H1N1 virus" and return the 1st result
concept1 = search({'string':'H1N1 virus', 'pageSize': 1})
print(concept1)

{'classType': 'searchResults', 'results': [{'ui': 'C1615607', 'rootSource': 'MTH', 'uri': 'https://uts-ws.nlm.nih.gov/rest/content/2019AB/CUI/C1615607', 'name': 'Influenza A Virus, H1N1 Subtype'}]}


In [46]:
# Get its definition in "MSH"
definition1 = definition({'sabs': 'MSH', 'pageSize': 1}, concept1["results"][0]["ui"])
print(definition1[0]["value"])

Any viral organism that can be assigned to the Influenza A species subtype H1N1.


In [49]:
# Get its semantic type
information1 = information({}, concept1["results"][0]["ui"])
print(information1["semanticTypes"])

[{'name': 'Virus', 'uri': 'https://uts-ws.nlm.nih.gov/rest/semantic-network/2019AB/TUI/T005'}]


In [56]:
# Get its synonyms
atoms1 = atoms({'language':'ENG', 'pageSize':100}, concept1["results"][0]["ui"])
print([atom["name"] for atom in atoms1])

['H1N1 Virus', 'H1N1 Viruses', 'INFLUENZA A VIRUS SUBTYPE H1N1', 'Influenza A (H1N1)', 'Influenza A (H1N1) Virus', 'Influenza A (H1N1) virus', 'Influenza A Virus H1N1 Subtype', 'Influenza A Virus, H1N1 Subtype', 'Influenza A Virus, H1N1 Subtype', 'Influenza A virus subtype H1N1', 'Influenza A virus subtype H1N1 (organism)', 'Influenza virus A (H1N1)', 'Swine Influenza A (H1N1) Virus', 'Swine-origin influenza A (H1N1) virus', 'Virus, H1N1', 'Viruses, H1N1']


In [83]:
# Get its parents in SNOMEDCT_US
# Not showing Obsolete/Suppressible atoms

atoms2 = atoms({'sabs': 'SNOMEDCT_US', 'pageSize':100}, concept1["results"][0]["ui"])
parents1 = parents({"pageSize":100}, atoms2[0]["code"])
for a_parent in parents1:
    print(a_parent["name"])

Influenza A virus subtype H1
Influenza A virus subtype N1


In [91]:
# Get its children in SNOMEDCT_US
# Not showing Obsolete/Suppressible atoms

children1 = children({"pageSize":100, "includeObsolete":"true", "includeSuppressible":"true"}, atoms2[0]["code"])
for a_child in children1:
    print(a_child["name"])

Influenza A virus A/Leningrad/621/86 (H1N1)
Influenza A virus A/Leningrad/624/86 (H1N1)
Pandemic influenza A (H1N1) 2009


In [93]:
# Get information about C0043031
information2 = information({}, 'C0043031')
print(information2)

{'classType': 'Concept', 'ui': 'C0043031', 'suppressible': False, 'dateAdded': '09-30-1990', 'majorRevisionDate': '02-21-2019', 'status': 'R', 'semanticTypes': [{'name': 'Organic Chemical', 'uri': 'https://uts-ws.nlm.nih.gov/rest/semantic-network/2019AB/TUI/T109'}, {'name': 'Pharmacologic Substance', 'uri': 'https://uts-ws.nlm.nih.gov/rest/semantic-network/2019AB/TUI/T121'}, {'name': 'Hazardous or Poisonous Substance', 'uri': 'https://uts-ws.nlm.nih.gov/rest/semantic-network/2019AB/TUI/T131'}], 'atomCount': 80, 'attributeCount': 0, 'cvMemberCount': 0, 'atoms': 'https://uts-ws.nlm.nih.gov/rest/content/2019AB/CUI/C0043031/atoms', 'definitions': 'https://uts-ws.nlm.nih.gov/rest/content/2019AB/CUI/C0043031/definitions', 'relations': 'https://uts-ws.nlm.nih.gov/rest/content/2019AB/CUI/C0043031/relations', 'defaultPreferredAtom': 'https://uts-ws.nlm.nih.gov/rest/content/2019AB/CUI/C0043031/atoms/preferred', 'relationCount': 34, 'name': 'Warfarin'}


In [94]:
# Get its definition in "CHV"
definition2 = definition({'sabs': 'CHV', 'pageSize': 1}, information2["ui"])
print(definition2[0]["value"])

a blood thinner


In [95]:
# Get its semantic type
print(information2["semanticTypes"])

[{'name': 'Organic Chemical', 'uri': 'https://uts-ws.nlm.nih.gov/rest/semantic-network/2019AB/TUI/T109'}, {'name': 'Pharmacologic Substance', 'uri': 'https://uts-ws.nlm.nih.gov/rest/semantic-network/2019AB/TUI/T121'}, {'name': 'Hazardous or Poisonous Substance', 'uri': 'https://uts-ws.nlm.nih.gov/rest/semantic-network/2019AB/TUI/T131'}]


In [99]:
# Get its synonyms
atoms3 = atoms({'language':'ENG', 'pageSize':100, 'sabs':'SNOMEDCT_US'}, information2["ui"])
print([atom["name"] for atom in atoms3])

['Product containing warfarin (medicinal product)', 'Warfarin', 'Warfarin (substance)', 'Warfarin-containing product']


In [100]:
#Get parents in USPMG
atoms4 = atoms({'sabs': 'USPMG', 'pageSize':100, includeObsolete="true", includeSuppressible="true"}, concept1["results"][0]["ui"])
parents2 = parents({"pageSize":100}, atoms4[0]["code"])
for a_parent in parents2:
    print(a_parent["name"])
    
#Errors: USPMG is not supported by the REST service

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [120]:
import mysql.connector
import pprint

# Connect to the database. 
# search_path is set to include mimiciii on the server side.
conn = mysql.connector.connect(host="172.16.35.100", port="3307", user="umls", password="umls", database="umls2019")
cur = conn.cursor()

In [121]:
def run_query(cur, query):
    cur.execute(query)
    query_result = cur.fetchall()
    col_names = [ col[0] for col in cur.description]
    pprint.pprint(col_names)
    pprint.pprint(query_result)

In [122]:
# SQL to get parent concepts from MySQL database
sql_query1 = """
select distinct b.STR from MRREL a, MRCONSO b WHERE CUI1='C0043031' and a.CUI2=b.CUI and REL='PAR' and a.SAB='USPMG' and LAT='ENG' and b.SAB='MTH'
"""

run_query(cur, sql_query1)

['STR']
[('Anticoagulants',)]


In [124]:
#SQL to get siblings concepts
sql_query2 = """
select distinct b.STR from MRREL a, MRCONSO b WHERE CUI1='C0043031' and a.CUI2=b.CUI and REL='SIB' and a.SAB='USPMG' and LAT='ENG' and b.SAB='MTH'
"""

run_query(cur, sql_query2)

['STR']
[('fondaparinux',), ('dabigatran',), ('Dalteparin',), ('rivaroxaban',)]
